In [ ]:
import numpy as np
import random
from sklearn.datasets import make_classification
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Diffie-Hellman parameters
q = 7919  # Small prime for demo; use large prime in real systems
alpha = 2  # Primitive root modulo q


def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)
        print(data.shape)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.2, random_state=42)

        self.model_weights = None
        self.intercept = 0

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.partial_fit(self.X_train, self.y_train,classes=[0,1])
        self.model_weights = clf.coef_[0]
        self.intercept = clf.intercept_[0]

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    # Load and preprocess dataset
    X, y = make_classification(n_samples=1000, n_features=30, n_classes=2,
                           n_informative=15, n_redundant=5, flip_y=0.1,
                           random_state=42)


    print(X.shape)
    # Normalize features
    

    # Split data across nodes
    split_data = np.array_split(X, num_nodes)
    split_labels = np.array_split(y, num_nodes)

    # Initialize nodes
    nodes = [Node(i, split_data[i], split_labels[i]) for i in range(num_nodes)]

    print(f"\n🎯 Initial Training at Node 0")
    nodes[0].train_model()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        shared_key = sender.compute_shared_key(receiver.public_key)
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key)
        print(f"Node {sender.id} encrypted model for Node {receiver.id} with shared key")

        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"Node {receiver.id} decrypted model")

        receiver.train_model()
        print(f"Node {receiver.id} trained model")

        current_holder = (current_holder + 1) % num_nodes

        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc}")

    return nodes

def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)

(1000, 30)
(334, 30)
(333, 30)
(333, 30)

🎯 Initial Training at Node 0
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 1
Node 0 Test Accuracy: 0.7164179104477612
Node 1 Test Accuracy: 0.7164179104477612
Node 2 Test Accuracy: 0.7611940298507462
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted model
Node 1 trained model
Node 1 encrypted model for Node 2 with shared key
Node 2 decrypted model
Node 2 trained model
Node 2 encrypted model for Node 0 with shared key
Node 0 decrypted model
Node 0 trained model

📊 Accuracy after Round 2
Node 0 Test Accuracy: 0.7014925373134329
Node 1 Test Accuracy: 0.7611940298507462
Node 2 Test Accuracy: 0.7611940298507462
Node 0 encrypted model for Node 1 with shared key
Node 1 decrypted 

In [3]:
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Diffie-Hellman params
q = 7919
alpha = 2

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

        self.model_weights = None
        self.intercept = 0

        # For EWC
        self.ewc_lambda = 100  # EWC penalty strength
        self.ewc_importance = None
        self.optimal_weights = None

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def estimate_importance(self):
        """Estimate Fisher Information (squared gradients) after training on Node 0."""
        clf = SGDClassifier(loss='log_loss', max_iter=1, learning_rate='constant', eta0=0.01, random_state=42)
        clf.partial_fit(self.X_train, self.y_train, classes=[0, 1])
        grads = clf.coef_[0]  # Use gradients as proxy for Fisher
        self.ewc_importance = grads ** 2
        self.optimal_weights = np.copy(self.model_weights)

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        clf.partial_fit(self.X_train, self.y_train, classes=[0, 1])
        updated_weights = clf.coef_[0]
        updated_intercept = clf.intercept_[0]

        # Apply EWC regularization if applicable
        if self.ewc_importance is not None and self.optimal_weights is not None:
            # Penalize deviation from original weights
            penalty = self.ewc_lambda * self.ewc_importance * (updated_weights - self.optimal_weights) ** 2
            updated_weights -= 0.01 * penalty  # Adjust learning

        self.model_weights = updated_weights
        self.intercept = updated_intercept

    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def load_and_prepare(filepath):
    df = pd.read_csv(filepath)

    # Rename last column to 'target'
    df.columns.values[-1] = 'target'

    # Encode 'yes' → 1 and 'no' → 0
    #df['target'] = df['target'].map({'YES': 1, 'NO': 0})

    y = df['target'].values
    X = df.drop(columns=['target']).values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y




def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    # Load datasets for each node
    heart_X, heart_y = load_and_prepare('heart_dataset_1000.csv')
    kidney_X, kidney_y = load_and_prepare('kidney_dataset_1000.csv')
    lung_X, lung_y = load_and_prepare('lung_dataset_1000.csv')

    # Assign to nodes
    datasets = [(heart_X, heart_y), (kidney_X, kidney_y), (lung_X, lung_y)]
    nodes = [Node(i, datasets[i][0], datasets[i][1]) for i in range(num_nodes)]

    # Step 1: Let only Node 0 train and estimate importance
    print(f"\n🧠 Initial training on Node 0")
    nodes[0].train_model()
    print(f"Node 0 Initial Weights: {nodes[0].model_weights}, Intercept: {nodes[0].intercept}")
    nodes[0].estimate_importance()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        print(f"\n🔄 Transfer {t + 1} (Round {(t // num_nodes) + 1})")

        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        # Compute shared keys
        shared_key_send = sender.compute_shared_key(receiver.public_key)
        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        print(f"🔐 Shared key (Sender Node {sender.id} → Receiver Node {receiver.id}): {shared_key_send}")
        assert shared_key_send == shared_key_recv, "Shared keys do not match!"

        # Encrypt model
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key_send)
        print(f"🔒 Encrypted weights: {enc_weights}")
        print(f"🔒 Encrypted intercept: {enc_intercept}")

        # Decrypt model
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"🔓 Decrypted weights at Node {receiver.id}: {receiver.model_weights}")
        print(f"🔓 Decrypted intercept at Node {receiver.id}: {receiver.intercept}")

        # Save copy before training to see weight change
        pre_train_weights = receiver.model_weights.copy()
        pre_train_intercept = receiver.intercept

        # Train
        receiver.train_model()

        # Show weight updates
        print(f"✅ Node {receiver.id} Weights After Training: {receiver.model_weights}")
        print(f"✅ Node {receiver.id} Intercept After Training: {receiver.intercept}")

        # Print EWC penalty (if applicable)
        if receiver.ewc_importance is not None and receiver.optimal_weights is not None:
            penalty = receiver.ewc_lambda * receiver.ewc_importance * (pre_train_weights - receiver.optimal_weights) ** 2
            print(f"📎 EWC Penalty Applied: {penalty}")

        # Estimate importance only once
        if receiver.ewc_importance is None:
            receiver.estimate_importance()
            print(f"📊 Node {receiver.id} estimated importance (Fisher Information).")

        print(f"🎯 Node {receiver.id} completed training.")

        current_holder = (current_holder + 1) % num_nodes

        # Evaluation after each full round
        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc}")

    return nodes


def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)



🧠 Initial training on Node 0
Node 0 Initial Weights: [ 0.08899143 -0.01524465  0.59153917  0.62350418 -0.04589035  0.53908453
  0.87835249], Intercept: -0.8776958498390589

🔄 Transfer 1 (Round 1)
🔐 Shared key (Sender Node 0 → Receiver Node 1): 5244
🔒 Encrypted weights: [np.float64(5244.088991426288), np.float64(5243.984755353062), np.float64(5244.591539174428), np.float64(5244.6235041769), np.float64(5243.95410964984), np.float64(5244.539084532885), np.float64(5244.878352488798)]
🔒 Encrypted intercept: 5243.122304150161
🔓 Decrypted weights at Node 1: [ 0.08899143 -0.01524465  0.59153917  0.62350418 -0.04589035  0.53908453
  0.87835249]
🔓 Decrypted intercept at Node 1: -0.877695849839256
✅ Node 1 Weights After Training: [ 0.00808695  0.33514891 -0.27406065  0.6959853   0.52536123 -0.09007056
  0.61816961]
✅ Node 1 Intercept After Training: -1.606676805686837
📊 Node 1 estimated importance (Fisher Information).
🎯 Node 1 completed training.

🔄 Transfer 2 (Round 1)
🔐 Shared key (Sender Nod

In [6]:
import numpy as np
import pandas as pd
import random
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Diffie-Hellman params
q = 7919
alpha = 2

def modular_pow(base, exponent, modulus):
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

class Node:
    def __init__(self, id, data, labels):
        self.id = id
        self.private_key = random.randint(2, q - 2)
        self.public_key = modular_pow(alpha, self.private_key, q)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

        self.model_weights = None
        self.intercept = 0

        # For EWC
        self.ewc_lambda = 100  # EWC penalty strength
        self.ewc_importance = None
        self.optimal_weights = None

    def compute_shared_key(self, other_public_key):
        return modular_pow(other_public_key, self.private_key, q)

    def encrypt_model(self, weights, intercept, shared_key):
        enc_weights = [w + shared_key for w in weights]
        enc_intercept = intercept + shared_key
        return enc_weights, enc_intercept

    def decrypt_model(self, enc_weights, enc_intercept, shared_key):
        self.model_weights = np.array([w - shared_key for w in enc_weights])
        self.intercept = enc_intercept - shared_key

    def estimate_importance(self):
        """Estimate Fisher Information (squared gradients) after training on Node 0."""
        clf = SGDClassifier(loss='log_loss', max_iter=1, learning_rate='constant', eta0=0.01, random_state=42)
        clf.partial_fit(self.X_train, self.y_train, classes=[0, 1])
        grads = clf.coef_[0]  # Use gradients as proxy for Fisher
        self.ewc_importance = grads ** 2
        self.optimal_weights = np.copy(self.model_weights)

    def train_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01, random_state=42)
        
        if self.model_weights is not None:
            clf.coef_ = self.model_weights.reshape(1, -1)
            clf.intercept_ = np.array([self.intercept])
            clf.classes_ = np.array([0, 1])
        
        # Perform the training step
        clf.partial_fit(self.X_train, self.y_train, classes=[0, 1])
        
        updated_weights = clf.coef_[0]
        updated_intercept = clf.intercept_[0]

        # Keep a copy of weights before EWC, for verification
        weights_before_ewc = updated_weights.copy()

        # Apply EWC regularization if applicable
        if self.ewc_importance is not None and self.optimal_weights is not None:
            delta = updated_weights - self.optimal_weights
            penalty = self.ewc_lambda * self.ewc_importance * (delta ** 2)
            correction = 0.01 * self.ewc_lambda * self.ewc_importance * delta
            updated_weights -= correction

            print(f"📎 EWC Penalty Applied: {penalty}")
            print(f"🆚 Weights Before EWC: {weights_before_ewc}")
            print(f"✅ Weights After EWC: {updated_weights}")
            print(f"↕️ Weight Change Due to EWC: {weights_before_ewc - updated_weights}")

        # Update final weights and intercept
        self.model_weights = updated_weights
        self.intercept = updated_intercept


    def evaluate(self):
        clf = SGDClassifier()
        clf.coef_ = self.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([self.intercept])
        clf.classes_ = np.array([0, 1])
        preds = clf.predict(self.X_test)
        return accuracy_score(self.y_test, preds)

def load_and_prepare(filepath):
    df = pd.read_csv(filepath)

    # Rename last column to 'target'
    df.columns.values[-1] = 'target'

    # Encode 'yes' → 1 and 'no' → 0
    #df['target'] = df['target'].map({'YES': 1, 'NO': 0})

    y = df['target'].values
    X = df.drop(columns=['target']).values

    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X, y




def secure_gossip_training_dh(num_nodes=3, num_rounds=3):
    # Load datasets for each node
    heart_X, heart_y = load_and_prepare('heart_dataset_1000.csv')
    kidney_X, kidney_y = load_and_prepare('kidney_dataset_1000.csv')
    lung_X, lung_y = load_and_prepare('lung_dataset_1000.csv')

    # Assign to nodes
    datasets = [(heart_X, heart_y), (kidney_X, kidney_y), (lung_X, lung_y)]
    nodes = [Node(i, datasets[i][0], datasets[i][1]) for i in range(num_nodes)]

    # Step 1: Let only Node 0 train and estimate importance
    print(f"\n🧠 Initial training on Node 0")
    nodes[0].train_model()
    print(f"Node 0 Initial Weights: {nodes[0].model_weights}, Intercept: {nodes[0].intercept}")
    nodes[0].estimate_importance()

    current_holder = 0
    total_transfers = num_rounds * num_nodes

    for t in range(total_transfers):
        print(f"\n🔄 Transfer {t + 1} (Round {(t // num_nodes) + 1})")

        sender = nodes[current_holder]
        receiver = nodes[(current_holder + 1) % num_nodes]

        # Compute shared keys
        shared_key_send = sender.compute_shared_key(receiver.public_key)
        shared_key_recv = receiver.compute_shared_key(sender.public_key)
        print(f"🔐 Shared key (Sender Node {sender.id} → Receiver Node {receiver.id}): {shared_key_send}")
        assert shared_key_send == shared_key_recv, "Shared keys do not match!"

        # Encrypt model
        enc_weights, enc_intercept = sender.encrypt_model(sender.model_weights, sender.intercept, shared_key_send)
        print(f"🔒 Encrypted weights: {enc_weights}")
        print(f"🔒 Encrypted intercept: {enc_intercept}")

        # Decrypt model
        receiver.decrypt_model(enc_weights, enc_intercept, shared_key_recv)
        print(f"🔓 Decrypted weights at Node {receiver.id}: {receiver.model_weights}")
        print(f"🔓 Decrypted intercept at Node {receiver.id}: {receiver.intercept}")

        # Save copy before training to see weight change
        pre_train_weights = receiver.model_weights.copy()
        pre_train_intercept = receiver.intercept

        # Train
        receiver.train_model()

        # Show weight updates
        print(f"✅ Node {receiver.id} Weights After Training: {receiver.model_weights}")
        print(f"✅ Node {receiver.id} Intercept After Training: {receiver.intercept}")

        # Print EWC penalty (if applicable)
        if receiver.ewc_importance is not None and receiver.optimal_weights is not None:
            penalty = receiver.ewc_lambda * receiver.ewc_importance * (pre_train_weights - receiver.optimal_weights) ** 2
            print(f"📎 EWC Penalty Applied: {penalty}")

        # Estimate importance only once
        if receiver.ewc_importance is None:
            receiver.estimate_importance()
            print(f"📊 Node {receiver.id} estimated importance (Fisher Information).")

        print(f"🎯 Node {receiver.id} completed training.")

        current_holder = (current_holder + 1) % num_nodes

        # Evaluation after each full round
        if (t + 1) % num_nodes == 0:
            print(f"\n📊 Accuracy after Round {(t + 1) // num_nodes}")
            for node in nodes:
                acc = node.evaluate()
                print(f"Node {node.id} Test Accuracy: {acc}")

    return nodes


def evaluate_nodes(nodes):
    print("\n📊 Final Evaluation Results:")
    for node in nodes:
        acc = node.evaluate()
        print(f"Node {node.id} Test Accuracy: {acc}")

# Run
nodes = secure_gossip_training_dh()
evaluate_nodes(nodes)



🧠 Initial training on Node 0
Node 0 Initial Weights: [ 0.08899143 -0.01524465  0.59153917  0.62350418 -0.04589035  0.53908453
  0.87835249], Intercept: -0.8776958498390589

🔄 Transfer 1 (Round 1)
🔐 Shared key (Sender Node 0 → Receiver Node 1): 5786
🔒 Encrypted weights: [np.float64(5786.088991426288), np.float64(5785.984755353062), np.float64(5786.591539174428), np.float64(5786.6235041769), np.float64(5785.95410964984), np.float64(5786.539084532885), np.float64(5786.878352488798)]
🔒 Encrypted intercept: 5785.122304150161
🔓 Decrypted weights at Node 1: [ 0.08899143 -0.01524465  0.59153917  0.62350418 -0.04589035  0.53908453
  0.87835249]
🔓 Decrypted intercept at Node 1: -0.877695849839256
✅ Node 1 Weights After Training: [ 0.00808695  0.33514891 -0.27406065  0.6959853   0.52536123 -0.09007056
  0.61816961]
✅ Node 1 Intercept After Training: -1.606676805686837
📊 Node 1 estimated importance (Fisher Information).
🎯 Node 1 completed training.

🔄 Transfer 2 (Round 1)
🔐 Shared key (Sender Nod